In [ ]:
import numpy as np
import pandas as pd
import pickle
from gurobipy import *
from Data import Data
from Solver import Solver
from InventorySimulator import InventorySimulator
from Simulator import Simulator
from X_finder import X_finder

In [ ]:
# find X for FCT | Objective = min sum(X) EXT_1

with open('dic_fct.pickle', 'rb') as handle:
    dic = pickle.load(handle)
    
for ii in range(45,46):
    for jj in range(6,7):
        print(ii , jj)
        X = X_finder(test_num=(ii,jj), dic=dic)
        file = './fct_all/' + str(ii) + '_' + str(jj) + '.x'
        np.savetxt(file, X.det_x, fmt='%d')

In [ ]:
df = pd.read_csv('./FCT_all.csv')
opt = {}
for r in range(len(df)):
    opt[(df.iloc[r]['i'], df.iloc[r]['j'])] = df.iloc[r]['lb']

In [ ]:
# simulate FCT
dic = {}
for ii in range(45,46):
    for jj in range(6,7):
        data = Data((ii,jj))
        data.gen_scn(20, 2000)
        file = './fct_all/' + str(ii) + '_' + str(jj) + '.x'
        X = Simulator(model_type='sfct', file=file, data=data)
        if X.status:
            dic[(ii,jj)] = [X.obj]
            print('{}\t{}\t{}\t{}\t{}'.format(ii, jj, opt[ii,jj], X.obj, round(X.time, 4)))
        else:
            dic[(ii,jj)] = ['{}\t{}\t{}\t*************'.format(ii, jj, opt[ii,jj])]

In [ ]:
pd.DataFrame(dic).T.to_csv('./FCT_all.csv')